In [2]:
import pandas as pd
import numpy as np
from tabpfn import TabPFNRegressor
from sklearn.metrics import r2_score, mean_squared_error

# 1. Load
train = pd.read_csv('train.csv')
val   = pd.read_csv('val.csv')
test  = pd.read_csv('test.csv')

In [3]:
from sklearn.metrics import r2_score, root_mean_squared_error

In [4]:

# 1. Load raw CSVs
train = pd.read_csv('train.csv')
val   = pd.read_csv('val.csv')
test  = pd.read_csv('test.csv')

# 2. Impute missing values (TRAIN-only statistics)
mg_mode    = train['MG'].mode()[0]
lon_med    = train['Longitude'].median()
mean_cols  = ['Lodging','PlantHeight','SeedSize','Protein','Oil']
mean_vals  = train[mean_cols].mean()

for df in (train, val, test):
    df['MG']        = df['MG'].fillna(mg_mode)
    df['Longitude'] = df['Longitude'].fillna(lon_med)
    for c in mean_cols:
        df[c]      = df[c].fillna(mean_vals[c])

# 3. Feature definitions
temporal_feats = ['MaxTemp','MinTemp','AvgTemp','AvgHumidity','Precipitation','Radiation']
static_feats   = ['Latitude','Longitude','Row.Spacing']
plant_feats    = ['Lodging','PlantHeight','SeedSize','Protein','Oil']
cluster_feats  = [f'Cluster_{i}' for i in range(40)]

# 4. Aggregation function
def aggregate_sequences(df, target='Yield', agg_target='mean'):
    agg = {}
    # temporal: mean & std
    for f in temporal_feats:
        agg[f'{f}_mean'] = (f, 'mean')
        agg[f'{f}_std']  = (f, 'std')
    # static geography: first
    for f in static_feats:
        agg[f] = (f, 'first')
    # plant: MG by mode, others by first
    agg['MG'] = ('MG', lambda x: x.mode().iloc[0])
    for f in plant_feats:
        agg[f] = (f, 'first')
    # clusters: mean & std
    for f in cluster_feats:
        agg[f'{f}_mean'] = (f, 'mean')
        agg[f'{f}_std']  = (f, 'std')
    # target
    if agg_target=='mean':
        agg[target] = (target, 'mean')
    else:
        agg[target] = (target, lambda x: x.iloc[-1])
    return df.groupby('TimeSeriesLabel').agg(**agg).reset_index(drop=True)

train_agg = aggregate_sequences(train)
val_agg   = aggregate_sequences(val)
test_agg  = aggregate_sequences(test)

# 5. Split into X/y
X_train = train_agg.drop('Yield', axis=1)
y_train = train_agg['Yield']

X_val   = val_agg.drop('Yield',   axis=1)
y_val   = val_agg['Yield']

X_test  = test_agg.drop('Yield',  axis=1)
y_test  = test_agg['Yield']


In [5]:
# 6. Quantization helpers
def fit_quantizer(df, b):
    levels = 2**b
    params = {}
    for col in df.columns:
        xmin, xmax = df[col].min(), df[col].max()
        if xmax==xmin: continue
        delta = (xmax-xmin)/(levels-1)
        params[col] = (xmin, delta)
    return params

def apply_quantizer(df, params):
    df_q = df.copy()
    for col, (xmin, delta) in params.items():
        df_q[col] = xmin + delta * np.round((df_q[col]-xmin)/delta)
    return df_q

In [6]:
b = 8
q_params = fit_quantizer(X_train, b)

Xtr_q = apply_quantizer(X_train, q_params).to_numpy()
Xvl_q = apply_quantizer(X_val,   q_params).to_numpy()
Xte_q = apply_quantizer(X_test,  q_params).to_numpy()

# Subsample up to 10k for fitting (reproducible)
n_sub = min(10_000, len(Xtr_q))
rng = np.random.RandomState(42)
idx = rng.choice(len(Xtr_q), size=n_sub, replace=False)
Xsub, ysub = Xtr_q[idx], y_train.to_numpy()[idx]

model = TabPFNRegressor()
model.fit(Xsub, ysub)

# Evaluate
for name, Xq, y in [
    ('Val',  Xvl_q, y_val.to_numpy()),
    ('Test', Xte_q, y_test.to_numpy())
]:
    preds = model.predict(Xq)
    print(f"b={b} → {name} R²: {r2_score(y, preds):.4f}, "
          f"RMSE: {root_mean_squared_error(y, preds):.4f}")



b=8 → Val R²: 0.7772, RMSE: 7.0752
b=8 → Test R²: 0.7790, RMSE: 6.9979


In [7]:
# 8. Baseline: Real MLP for tabular regression (PyTorch preferred, sklearn fallback)

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

# robust RMSE import (sklearn versions differ)
try:
    from sklearn.metrics import root_mean_squared_error as _rmse
    def RMSE(y, p): return _rmse(y, p)
except Exception:
    def RMSE(y, p): return mean_squared_error(y, p, squared=False)

# -----------------------------
# Try PyTorch first
# -----------------------------
_USE_TORCH = True
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
except Exception:
    _USE_TORCH = False

def mlp_eval_print(name, y_true, y_pred):
    print(f"{name} R²: {r2_score(y_true, y_pred):.4f}, RMSE: {RMSE(y_true, y_pred):.4f}")

if _USE_TORCH:
    # ---- PyTorch MLP ----
    class MLP(nn.Module):
        def __init__(self, d_in, widths=(512, 256, 128), dropout=0.1):
            super().__init__()
            layers = []
            prev = d_in
            for w in widths:
                layers += [nn.Linear(prev, w), nn.ReLU(), nn.Dropout(dropout)]
                prev = w
            layers += [nn.Linear(prev, 1)]
            self.net = nn.Sequential(*layers)

        def forward(self, x):
            return self.net(x).squeeze(-1)

    def train_mlp_torch(Xtr, ytr, Xvl, yvl, *,
                        epochs=200, lr=1e-3, batch_size=1024, patience=20, seed=42):
        torch.manual_seed(seed)
        np.random.seed(seed)

        scaler = StandardScaler().fit(Xtr)
        Xtr_s = scaler.transform(Xtr).astype(np.float32)
        Xvl_s = scaler.transform(Xvl).astype(np.float32)

        Xtr_t = torch.from_numpy(Xtr_s)
        ytr_t = torch.from_numpy(ytr.astype(np.float32))
        Xvl_t = torch.from_numpy(Xvl_s)
        yvl_t = torch.from_numpy(yvl.astype(np.float32))

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = MLP(d_in=Xtr_t.shape[1]).to(device)
        opt = optim.AdamW(model.parameters(), lr=lr)
        loss_fn = nn.MSELoss()

        def batches(X, y, bs):
            idx = np.random.permutation(len(X))
            for i in range(0, len(X), bs):
                j = idx[i:i+bs]
                yield X[j], y[j]

        best_loss = float('inf')
        best_state = None
        bad = 0

        for ep in range(1, epochs+1):
            model.train()
            for xb, yb in batches(Xtr_t, ytr_t, batch_size):
                xb, yb = xb.to(device), yb.to(device)
                opt.zero_grad()
                pred = model(xb)
                loss = loss_fn(pred, yb)
                loss.backward()
                opt.step()

            # val
            model.eval()
            with torch.no_grad():
                val_pred = model(Xvl_t.to(device)).cpu()
                val_loss = loss_fn(val_pred, yvl_t).item()

            if val_loss < best_loss - 1e-6:
                best_loss = val_loss
                best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
                bad = 0
            else:
                bad += 1
                if bad >= patience:
                    break

        if best_state is not None:
            model.load_state_dict(best_state)

        return model, scaler, device

    def predict_mlp_torch(model, scaler, device, X):
        Xs = scaler.transform(X).astype(np.float32)
        Xt = torch.from_numpy(Xs).to(device)
        model.eval()
        with torch.no_grad():
            preds = model(Xt).cpu().numpy()
        return preds

else:
    # ---- sklearn fallback ----
    from sklearn.neural_network import MLPRegressor

    def train_mlp_sklearn(Xtr, ytr, Xvl, yvl, *, seed=42):
        # Fit scaler on train, apply to all
        scaler = StandardScaler().fit(Xtr)
        Xtr_s = scaler.transform(Xtr)
        Xvl_s = scaler.transform(Xvl)

        # Early stopping internal to sklearn (uses a split from training data)
        mlp = MLPRegressor(hidden_layer_sizes=(512, 256, 128),
                           activation='relu',
                           solver='adam',
                           max_iter=500,
                           learning_rate_init=1e-3,
                           early_stopping=True,
                           n_iter_no_change=20,
                           random_state=seed)
        mlp.fit(Xtr_s, ytr)
        return mlp, scaler

    def predict_mlp_sklearn(model, scaler, X):
        return model.predict(scaler.transform(X))

# -----------------------------
# Run MLP over the same bit-depths
# -----------------------------
print("\n===== Real MLP Baseline =====")
rng = np.random.RandomState(42)

for b in [2,4,6,8,16,32,64,128,256]:
    # Quantize using your existing helpers/params
    q_params = fit_quantizer(X_train, b)
    Xtr_q = apply_quantizer(X_train, q_params).to_numpy()
    Xvl_q = apply_quantizer(X_val,   q_params).to_numpy()
    Xte_q = apply_quantizer(X_test,  q_params).to_numpy()

    # Subsample 10k for parity with TabPFN
    n_sub = min(10_000, len(Xtr_q))
    idx = rng.choice(len(Xtr_q), size=n_sub, replace=False)
    Xsub, ysub = Xtr_q[idx], y_train.to_numpy()[idx]

    print(f"\nb={b}")
    if _USE_TORCH:
        model, scaler, device = train_mlp_torch(Xsub, ysub, Xvl_q, y_val.to_numpy(),
                                                epochs=200, lr=1e-3, batch_size=1024, patience=20)
        pred_v = predict_mlp_torch(model, scaler, device, Xvl_q)
        pred_t = predict_mlp_torch(model, scaler, device, Xte_q)
    else:
        model, scaler = train_mlp_sklearn(Xsub, ysub, Xvl_q, y_val.to_numpy())
        pred_v = predict_mlp_sklearn(model, scaler, Xvl_q)
        pred_t = predict_mlp_sklearn(model, scaler, Xte_q)

    mlp_eval_print("Val ", y_val.to_numpy(), pred_v)
    mlp_eval_print("Test", y_test.to_numpy(), pred_t)


===== Real MLP Baseline =====

b=2
Val  R²: -1.0954, RMSE: 21.6975
Test R²: -1.9444, RMSE: 25.5457

b=4
Val  R²: 0.5136, RMSE: 10.4539
Test R²: 0.5296, RMSE: 10.2111

b=6
Val  R²: 0.5120, RMSE: 10.4705
Test R²: 0.5246, RMSE: 10.2650

b=8
Val  R²: 0.4945, RMSE: 10.6576
Test R²: 0.5080, RMSE: 10.4423

b=16
Val  R²: 0.5003, RMSE: 10.5953
Test R²: 0.5153, RMSE: 10.3651

b=32
Val  R²: 0.4794, RMSE: 10.8153
Test R²: 0.4946, RMSE: 10.5842

b=64
Val  R²: 0.5123, RMSE: 10.4679
Test R²: 0.5196, RMSE: 10.3189

b=128
Val  R²: 0.4976, RMSE: 10.6243
Test R²: 0.5231, RMSE: 10.2804

b=256
Val  R²: 0.5219, RMSE: 10.3639
Test R²: 0.5368, RMSE: 10.1318
